# Цель занятия
На этом занятии мы рассмотрим применение случайного леса.

**Случайный лес**
Случайный лес (Random Forest) — это ансамблевый метод машинного обучения, основанный на деревьях решений. Он использует множество деревьев решений для прогнозирования целевой переменной, а затем усредняет их ответы, чтобы получить более стабильный и точный результат. Каждое дерево в случайном лесе обучается на случайной подвыборке объектов и случайном подмножестве признаков, что делает модель менее склонной к переобучению. Кроме того, случайный лес может использоваться для оценки важности признаков, что позволяет определить наиболее значимые факторы для прогнозирования целевой переменной.

Датасет "Титаник" является одним из самых популярных в машинном обучении и содержит информацию о пассажирах парохода "Титаник".

Датасет содержит следующие признаки для каждого пассажира:

- Survived: показатель, выжил ли пассажир или нет (0 - не выжил, 1 - выжил);
- Pclass: класс пассажира (1 - первый, 2 - второй, 3 - третий);
- Name: имя пассажира;
- Sex: пол пассажира;
- Age: возраст пассажира в годах;
- SibSp: количество братьев/сестер или супругов на борту;
- Parch: количество родителей или детей на борту;
- Ticket: номер билета;
- Fare: стоимость билета;
- Cabin: номер каюты;
- Embarked: порт посадки (C - Шербур, Q - Квинстаун, S - Саутгемптон).

Для примера случайного леса на датасете Титаник мы будем использовать следующие шаги:

- Загрузим датасет Титаник и подготовим данные для обучения модели.
- Разделим данные на тренировочную и тестовую выборки.
- Определим настройки для случайного леса.
- Обучим модель на тренировочной выборке.
- Оценим качество модели на тестовой выборке.
- Выведем на экран важность признаков.

In [11]:
# Импортируем необходимые библиотеки и загрузим датасет:

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

# загрузка датасета
df = pd.read_csv('data/titanic.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'PassengerId'})
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
# Произведем предобработку данных, удалим ненужные столбцы, заменим пропущенные 
# значения и преобразуем категориальные признаки в числовые:

# удаление ненужных столбцов
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [16]:
# заполнение пропущенных значений
category_col = ["Sex", "Embarked"]
for col in df.columns:
    if col in category_col:
        df[col] = df[col].fillna(df[col].mode()[0])
    else:
        df[col] = df[col].fillna(df[col].mean())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [17]:
# преобразование категориальных признаков в числовые
df = pd.get_dummies(df, columns=["Sex", "Embarked"])
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,False,True,False,False,True
1,1,1,38.0,1,0,71.2833,True,False,True,False,False
2,1,3,26.0,0,0,7.9250,True,False,False,False,True
3,1,1,35.0,1,0,53.1000,True,False,False,False,True
4,0,3,35.0,0,0,8.0500,False,True,False,False,True


In [18]:
# Разделим данные на тренировочную и тестовую выборки:

X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Определим настройки для случайного леса и обучим модель на тренировочной выборке:

# определение настроек для случайного леса
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

# обучение модели на тренировочной выборке
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=42)

In [20]:
# Оценим качество модели на тестовой выборке:

# предсказание на тестовой выборке
y_pred = rf.predict(X_test)

# оценка качества модели на тестовой выборке
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print("Accuracy:", acc)
print("Confusion matrix:\n", cm)

Accuracy: 0.8100558659217877
Confusion matrix:
 [[93 12]
 [22 52]]


In [21]:
# Выведем на экран важность признаков:

# вывод важности признаков
importances = rf.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance}")

Pclass: 0.13141401307718267
Age: 0.1008043843049685
SibSp: 0.04446070417029835
Parch: 0.02940916134497456
Fare: 0.133319831842019
Sex_female: 0.2259090415793943
Sex_male: 0.29617801298154256
Embarked_C: 0.013341688442022635
Embarked_Q: 0.00904961582497526
Embarked_S: 0.016113546432622312
